In [116]:
# Buisness understanding...
#You are the sales manager for "BeerMart", an online beer store in the United States.  You want to build a recommendation
#system (collaborative) for your store, where customers will be recommended the beer that they are most likely to buy. 
#You collected the data about the ratings that the customers have provided in the past. You can download the dataset from the link below.

#Description: Each record is composed of a beer's name, the name of the user along with ratings provided by users. All ratings are on a scale from 1 to 5 with 5 being the best.
    
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [117]:
########Read the test and train data..
BeerDataDs=pd.read_csv('D:\\DataScience\\BeerRecommenderSystem\\beer_data.csv',skipinitialspace=True)
BeerDataDs.head()

,beer_beerid,review_profilename,review_overall
0,48215,stcules,3.0
1,52159,oline73,3.0
2,52159,alpinebryant,3.0
3,52159,rawthar,4.0
4,52159,RangerClegg,3.5


In [118]:
BeerDataDs.info()
### no misisng value...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475984 entries, 0 to 475983
Data columns (total 3 columns):
beer_beerid           475984 non-null int64
review_profilename    475884 non-null object
review_overall        475984 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 10.9+ MB


In [119]:
BeerDataDs.apply(lambda x:x.isnull().sum()) ## 100 misisng values in review_profilename    

beer_beerid             0
review_profilename    100
review_overall          0
dtype: int64

In [120]:
##3 Lets remove the blank values...
BeerDataDsDf=BeerDataDs[BeerDataDs!=""]
BeerDataDsDf.shape

(475984, 3)

In [121]:
(BeerDataDsDf).head()

,beer_beerid,review_profilename,review_overall
0,48215,stcules,3.0
1,52159,oline73,3.0
2,52159,alpinebryant,3.0
3,52159,rawthar,4.0
4,52159,RangerClegg,3.5


In [122]:
BeerDataDsDf.shape

(475984, 3)

In [123]:
# BeerDataDsDf
BeerDataDsDf1 = BeerDataDsDf.drop_duplicates(subset=['beer_beerid'], keep='first')

In [124]:
BeerDataDsDf1.shape  ###   total rows count 40308.. after removing the duplicates..

(40308, 3)

In [125]:
# #Lets find all distinct beers & total reviews they received
BeerDataDsDf1['beer_beerid'].nunique()  ## 40308 distict beers are rated which is about 42.6% of total enteries logged 94462
NoOfUsersReviewdBeer=BeerDataDsDf1.groupby('beer_beerid')['review_overall'].count().sort_values(ascending=False)
NoOfUsersReviewdBeer.head(5)

beer_beerid
77317    1
23569    1
23584    1
23581    1
23578    1
Name: review_overall, dtype: int64

In [126]:
NoOfUsersReviewdBeer.shape   ## 40308 enteries ...

(40308,)

In [127]:
# #Lets find all distinct beers & total reviews they received
#BeerDataDsDf1['beer_beerid'].nunique()  ## 40308 distict beers are rated which is about 42.6% of total enteries logged 94462
NoOfTimeTheBeerisReviewed=BeerDataDsDf1.groupby('beer_beerid')['review_overall'].count().sort_values(ascending=False)
NoOfTimeTheBeerisReviewed.head(5)

beer_beerid
77317    1
23569    1
23584    1
23581    1
23578    1
Name: review_overall, dtype: int64

In [128]:
## hpw mnay users have used the beer...
NoOfUsersUsedthisBeers=BeerDataDsDf1.groupby('beer_beerid')['review_profilename'].count().sort_values(ascending=False)
NoOfUsersUsedthisBeers.head(5)

beer_beerid
77317    1
23578    1
23590    1
23588    1
23587    1
Name: review_profilename, dtype: int64

In [129]:
BeerData=pd.merge(BeerDataDsDf1.reset_index(),NoOfTimeTheBeerisReviewed.reset_index(),on='beer_beerid',how='left')

In [130]:
BeerDataFinal=pd.merge(BeerData,NoOfUsersUsedthisBeers.reset_index(),on='beer_beerid',how='left')

In [131]:
BeerDataFinal.head()

,index,beer_beerid,review_profilename_x,review_overall_x,review_overall_y,review_profilename_y
0,0,48215,stcules,3.0,1,1
1,1,52159,oline73,3.0,1,1
2,5,58046,mikedrinksbeer2,4.5,1,1
3,12,436,Brent,3.0,1,1
4,18,25414,NJpadreFan,4.0,1,1


In [132]:
BeerDataFinal.columns=['index','beer_beerid','review_profilename','review_overall_rating','NoOfTimeTheBeerisReviewed','NoOfUsersUsedthisBeers']

In [133]:
BeerDataFinal.head()

,index,beer_beerid,review_profilename,review_overall_rating,NoOfTimeTheBeerisReviewed,NoOfUsersUsedthisBeers
0,0,48215,stcules,3.0,1,1
1,1,52159,oline73,3.0,1,1
2,5,58046,mikedrinksbeer2,4.5,1,1
3,12,436,Brent,3.0,1,1
4,18,25414,NJpadreFan,4.0,1,1


In [134]:
n_beers=BeerDataFinal.beer_beerid.unique().shape[0]

In [135]:
n_users=BeerDataFinal.review_profilename.unique().shape[0]

data_matrix = np.zeros((n_users, n_beers))
for line in BeerDataFinal.itertuples():
    data_matrix[line[1], line[2]] = line[3]

In [136]:
data_matrix = np.zeros((n_users, n_beers))

In [137]:
data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [138]:
BeerDataFinal.apply(lambda x:x.isnull().sum())
# TrainDs.apply(lambda x:x.isnull().sum())
BeerDataFinal=BeerDataFinal.dropna()

In [139]:
BeerDataFinal.head()

,index,beer_beerid,review_profilename,review_overall_rating,NoOfTimeTheBeerisReviewed,NoOfUsersUsedthisBeers
0,0,48215,stcules,3.0,1,1
1,1,52159,oline73,3.0,1,1
2,5,58046,mikedrinksbeer2,4.5,1,1
3,12,436,Brent,3.0,1,1
4,18,25414,NJpadreFan,4.0,1,1


In [140]:
# user item ratings to matrix form. It can be done using the pivot function in python.
#R=user item ratings to matrix form. It can be done using the pivot function in python
# R= np.array(ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0))
R=np.array(BeerDataFinal.pivot(index = 'review_profilename', columns ='beer_beerid', values = 'review_overall_rating').fillna(0))

In [142]:
class MF():

    # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
        self.sgd()
        mse = self.mse()
        training_process.append((i, mse))
        if (i+1) % 20 == 0:
            print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    # Ratings for user i and moive j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [144]:
mf = MF(R, K=20, alpha=0.001, beta=0.01, iterations=1)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
#print()


P x Q:
[[3.64645609 3.6352104  3.66814954 ... 3.64419368 3.65736555 3.62784213]
 [3.63574863 3.60212834 3.64758307 ... 3.63238231 3.62534308 3.63459778]
 [3.63583418 3.62262409 3.64458841 ... 3.64705933 3.64485626 3.63826204]
 ...
 [3.64477929 3.62878207 3.63075547 ... 3.64360288 3.63729836 3.63300749]
 [3.64879757 3.66364396 3.62490598 ... 3.6334435  3.64053317 3.6270617 ]
 [3.64802611 3.64027253 3.64281457 ... 3.6450641  3.64328225 3.61743934]]
